In [14]:
from transformers import pipeline
from transformers import DistilBertTokenizer
import json
import pandas as pd
import tqdm
import logger
import nltk
import matplotlib.pyplot as plt
import numpy as np
import regex
import dateparser
from utils.loader_findata import Finloader
import datetime

In [15]:
#def split_text_into_sentences(text):
#    sentences = nltk.sent_tokenize(text)
#    return sentences

#texts_data = []
#summarizer = pipeline("summarization", model="facebook/bart-large-cnn", min_length = 1)

#with open('../coindesk_news.json', 'r') as file:
#    loaded_data = json.load(file)

#results = []

In [16]:
#def split_text_into_sentences(text):
#    sentences = nltk.sent_tokenize(text)
#    return sentences

#texts_data = []
#summarizer = pipeline("summarization", model="facebook/bart-large-cnn", min_length = 1)
#
#with open('../coindesk_news.json', 'r') as file:
#    loaded_data = json.load(file)
#
#results = []
#
#for URL in tqdm.tqdm(list(loaded_data.keys())[:10]):
#    title = loaded_data[URL][0]
#    time = loaded_data[URL][1]
#    text = loaded_data[URL][2].replace('\n', '')
#    sequences = split_text_into_sentences(text)
#    res = ""
#    for i in range(len(sequences)):
#        res = res + summarizer(sequences[i], max_length = 10)[0]['summary_text']
#    texts_data.append([title, time, res])
#    results.append([len(text.split()), len(res.split())])
#print(*results, sep ='\n')

In [17]:
def extract_tokens(text: str):
    return set(regex.findall(pattern="[A-Z]{3,}", string=text))

def convert_time(text: str):
    raw_list = list(map(lambda p: p.replace('Updated', '').replace('Published', ''), text.split('UTC')))

    if len(raw_list) > 1:
        raw_list.pop(-1)
    
    raw_list = [dateparser.parse(i) for i in raw_list]

    try:
        return raw_list[0]
    except:
        print(text, raw_list)

def form_news_topandas(path_to_news_json:str, path_to_tokens:str):
    
    with open(path_to_tokens, 'r') as file:
        valid_tokens = set(file.read().split('\n'))
    with open(path_to_news_json, 'r') as file:
        loaded_data = json.load(file)
    
    res = []
    for URL in tqdm.tqdm(list(loaded_data.keys())[:]):
        title = loaded_data[URL][0]
        time = convert_time(loaded_data[URL][1])
        text = loaded_data[URL][2]
        tokens = list(extract_tokens(text) & valid_tokens)\
        
        res.append([title, time, text, ' '.join(tokens)])

    df_res =  pd.DataFrame(data = res, columns=['title', 'time', 'text', 'tokens']).sort_values('time')
    df_res=df_res.reset_index()
    df_res = df_res.drop(labels=['index'], axis = 1)
    return df_res
df_news = form_news_topandas('../../data/coindesk_news.json', '../../data/tokens_names.txt')

100%|██████████| 6863/6863 [00:07<00:00, 930.56it/s] 


In [18]:
loader = Finloader(r"C:\Users\Student\UPS2025\UPS2025_FinMat\data\clear_data")

filtered_loader_columns = []
loader_columns = loader.get_columns()
for i in loader_columns:
    if 'close_' in i:
        filtered_loader_columns.append(i.replace('close_', ''))

In [19]:
closes_data = pd.read_csv(r"../../data/clear_data/close.csv")
filtered_loader_columns.insert(0, 'openTime')
closes_data_filtered = closes_data[filtered_loader_columns]
closes_data_filtered = closes_data_filtered.set_index("openTime")
closes_data_filtered
closes_data_filtered = closes_data_filtered.pct_change()
closes_data_filtered = closes_data_filtered.iloc[1:]
closes_data = closes_data.index.astype(dtype=str)

In [20]:
def roundUpDateTime(dt):
    delta_5min = 5 - dt.minute % 5

    delta = datetime.timedelta(minutes = delta_5min)

    return datetime.datetime(dt.year, dt.month, dt.day,
                             dt.hour, dt.minute, dt.second) + delta

news_sentiment = []

for i in range(len(df_news)):
    finance_open_time = roundUpDateTime(dateparser.parse(str(df_news.iloc[i]['time'])))
    try:
        closes = closes_data_filtered.loc[finance_open_time.strftime(format="%Y-%m-%d %H:%M:%S+00:00")]
        row = [df_news.iloc[i]['text']]
        row.extend(list(closes))
        row.extend([df_news.iloc[i]['tokens']])
        news_sentiment.append(row)
    except:
        pass

dataset_cols = list(closes_data_filtered.columns)
dataset_cols.insert(0, 'news_text')
dataset_cols.append('token_names')
dataset_cols
dataset_raw = pd.DataFrame(data=news_sentiment, columns=dataset_cols)

In [21]:
dataset_dat = []

tokens = dataset_raw[list(set(dataset_raw.columns)-set(['news_text', 'token_names']))]

for i in range(len(dataset_raw)):
    row_tokens = tokens.iloc[i]
    news_text = dataset_raw.iloc[i]['news_text']
    token_names = dataset_raw.iloc[i]['token_names'].split()

    val = None
    token_names_cur = []
    if len(token_names) == 0:
        val = tokens.iloc[i].mean()
    else:
        token_vals = []
        token_names_cur = []
        for name in token_names:
            token_names_cur.extend(list(tokens.iloc[i].filter(like = name).index))
            token_vals.extend(list(tokens.iloc[i].filter(like = name).values))
        val = sum(token_vals)/len(token_vals)
    dataset_dat.append([news_text, val, ' '.join(token_names_cur)])

dataset = pd.DataFrame(dataset_dat, columns=['text', 'val', 'tokens'])

In [23]:
# Токенизация

#input_texts = []
#predicts = []
#tokens_lists = []
#
#for i in tqdm.tqdm(range(len(dataset))):
#    input_text = dataset.iloc[i]['text']
#    predict = dataset.iloc[i]['val']
#    token_list = dataset.iloc[i]['tokens']
#
#    for par in input_text.split('\n'):
#        input_texts.append(tokenizer(par, padding=True, truncation=True, return_tensors="pt"))
#        predicts.append(predict)
#        tokens_lists.append(token_list)

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

for param in model.base_model.parameters():
    param.requires_grad = False

NUM_EPOCHS = 10

import torch
from torch import nn

class RegressionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(768, 1)  # Один выход для регрессии
        self.dropout = nn.Dropout(p = 0.1, inplace= False)

    def forward(self, features):
        print(features.shape, "first")
        x = self.dropout(features)
        print(features.shape, "second")
        x = self.dense(x)
        print(features.shape, "third")
        return x


model.classifier = RegressionHead(model.config)
model.config.num_labels = 1

torch.Size([1, 2, 768]) first
torch.Size([1, 2, 768]) second
torch.Size([1, 2, 768]) third


TypeError: 'int' object is not callable

In [553]:
model.classifier

RegressionHead(
  (dense): Linear(in_features=768, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
plt.plot(train_losses_by_epochs)
plt.plot(val_losses_by_epochs)
plt.show()